📖 Background and Goal

Previously BI was configurated for shows transaction only at Analytics campaing level, some of the oldest perfomance campaings was configurated without utm at ad level (utm_content).
With the new BI and dashboards development it must needed to fix this in the transition until all these will be paused or just was one of these per campaign.

The data: 
 - G Analytics campaign transaction report
 - Dictionary Ad ID from FB Ads for matcc analytics Ad content with out data or ID term.

Key varaibles:
"Campaign" - utm_campaign from analytics transactions
"Ad Content	" - utm_content from analytics transactions
"Ad ID" -  Facebook ID using in Dictionary
"utm_campaign" - Facebook Campaign Name using in Dictionary

In [5]:
from datetime import datetime, timedelta
import seaborn as sns
import numpy as np
import glob
import pandas as pd

In [6]:
#Load analytics and utm fix Dict for asign FB Ad id to analytics Ad Content utm 
an = pd.read_csv('docs/csv campaignTransaction CHILE - CYC LATAM SEM 20220901-20220926.csv', sep=',', header=0)  
seg = pd.read_excel('docs/ID-campaigns_CL.xlsx', usecols=['utm_campaign','Ad name','Ad ID'])

#Convert Ad ID as string
seg['Ad ID'] = seg['Ad ID'].astype(str)

In [9]:
seg

,Ad ID,Ad name,utm_campaign
0,6231665086427,(not set),FB_CL_LK_IPP
1,6244339439427,(not set),FB_CL_CORE_Carreras_Online
2,6245873217627,(not set),FB_CL_RMKT_Carreras
3,246896626974,Buscador de Carreras,CYC_CL_Carreras_Online
4,401264627256,Buscas {Keyword:Cursos Online},CYC_CL_CURSOS_DISTANCIA
5,6255893940427,LK_DB_ad_serv_soc,FB_CL_LK_Conv
6,6255893940227,LK_DB_MP_CURSOS,FB_CL_LK_Conv


In [10]:
#merge both by Campaign and Ad content/Ad name
merge = pd.merge(an,seg, how="left", left_on=['Campaign','Ad Content'], right_on=['utm_campaign','Ad name'])

In [12]:
#filter by FB campaigns
merge[(merge['Campaign'].str.startswith('FB_CL', na=False)) & (merge['Ad ID'].notnull())].head()

,Transaction ID,Campaign,Ad Content,Medium,Google Ads Campaign ID,Google Ads Ad Group,Google Ads Ad Group ID,Transactions,Ad ID,Ad name,utm_campaign
0,FO-11893781,FB_CL_LK_Conv,LK_DB_ad_serv_soc,cpc,(not set),(not set),(not set),1,6255893940427,LK_DB_ad_serv_soc,FB_CL_LK_Conv
1,FO-11893825,FB_CL_LK_IPP,(not set),cpc,(not set),(not set),(not set),1,6231665086427,(not set),FB_CL_LK_IPP
2,FO-11893827,FB_CL_CORE_Carreras_Online,(not set),(not set),(not set),(not set),(not set),1,6244339439427,(not set),FB_CL_CORE_Carreras_Online
3,FO-11893901,FB_CL_LK_Conv,LK_DB_ad_serv_soc,cpc,(not set),(not set),(not set),1,6255893940427,LK_DB_ad_serv_soc,FB_CL_LK_Conv
4,FO-11893930,FB_CL_LK_Conv,LK_DB_MP_CURSOS,cpc,(not set),(not set),(not set),1,6255893940227,LK_DB_MP_CURSOS,FB_CL_LK_Conv


In [13]:
#merge in Ad ID NaN values the Ad Content Analytics values
merge["Ad ID"].fillna(merge["Ad Content"], inplace=True)
#drop dict and Original Ad content column
merge = merge.drop(['Ad Content','Ad name','utm_campaign'], 1)
#Rename Ad Id column with the original name for Analitycs
merge = merge.rename(columns={'Ad ID': 'Ad Content'}) 

C:\Users\Frank\AppData\Local\Temp/ipykernel_80188/2585063862.py:4: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  merge = merge.drop(['Ad Content','Ad name','utm_campaign'], 1)


In [15]:
merge[merge['Campaign'].str.startswith('FB_CL', na=False)].head()

,Transaction ID,Campaign,Medium,Google Ads Campaign ID,Google Ads Ad Group,Google Ads Ad Group ID,Transactions,Ad Content
0,FO-11893781,FB_CL_LK_Conv,cpc,(not set),(not set),(not set),1,6255893940427
1,FO-11893825,FB_CL_LK_IPP,cpc,(not set),(not set),(not set),1,6231665086427
2,FO-11893827,FB_CL_CORE_Carreras_Online,(not set),(not set),(not set),(not set),1,6244339439427
3,FO-11893901,FB_CL_LK_Conv,cpc,(not set),(not set),(not set),1,6255893940427
4,FO-11893930,FB_CL_LK_Conv,cpc,(not set),(not set),(not set),1,6255893940227


In [17]:
#Get dates from  first day of month and yesterday formatted
todayDate = datetime.today()
yesterdayDate = todayDate - timedelta(1)
firstDateOfMonthFormatted = yesterdayDate.replace(day=1).strftime('%Y-%m-%d')
yesterdayDateFormatted = (todayDate - timedelta(1)).strftime('%Y-%m-%d')

In [18]:
#Csv with location and name with dates
name = r'C:\power_bi\BI_SEM_2022\CL\Analytics\csv campaignTransaction CHILE - CYC LATAM SEM '+  firstDateOfMonthFormatted + ' - ' + yesterdayDateFormatted + '-' + '.csv'

#df export to csv and folder location"
merge.to_csv(name, sep=',', encoding='utf-8')

OUTPUT:
- After validate with final report and BI we proceed to include as a function in the Analytics daily reporting script and config.json.

In [ ]:
def fix_csv(sourceFile, diccFile):
  logging.info('Fix report '+sourceFile+' <> '+diccFile)

  an = pd.read_csv(sourceFile, sep=',', header=0)  
  seg = pd.read_excel(diccFile, usecols=['utm_campaign','Ad name','Ad ID'])

  seg['Ad ID'] = seg['Ad ID'].astype(str)
  merge = pd.merge(an,seg, how="left", left_on=['Campaign','Ad Content'], right_on=['utm_campaign','Ad name'])

  merge["Ad ID"].fillna(merge["Ad Content"], inplace=True)
  merge = merge.drop(['Ad Content','Ad name','utm_campaign'], 1)
  merge = merge.rename(columns={'Ad ID': 'Ad Content'}) 
  merge = merge[['Transaction ID','Campaign','Ad Content','Medium','Google Ads Campaign ID', 'Google Ads Ad Group','Google Ads Ad Group ID','Transactions']]

  merge.to_csv(sourceFile, index=False)